In [ ]:
# =============================================================================
# Model Integration - Multi-Provider Support (LangChain 1.0+)
# =============================================================================
"""
=====================================================================
Model Integration - Working with Multiple LLM Providers
=====================================================================

LangChain 1.0+ provides a unified interface to work with different
LLM providers through init_chat_model() - a universal model initializer.

Supported Providers:
--------------------
- OpenAI (GPT-4, GPT-4o, GPT-4.1, etc.)
- Anthropic (Claude 3, Claude 3.5, Claude 4)
- Google (Gemini Pro, Gemini Ultra)
- Groq (Llama, Mixtral, Qwen with ultra-fast inference)
- Cohere, Mistral, and more...

Key Concepts:
-------------
1. init_chat_model() - Universal model initialization
2. Streaming responses
3. Provider-specific features
4. Model switching without code changes

Architecture:
-------------
    ┌─────────────────────────────────────┐
    │         init_chat_model()           │
    │     (Universal Interface)           │
    └─────────────────────────────────────┘
                    │
        ┌───────────┼───────────┐
        │           │           │
        ▼           ▼           ▼
    ┌───────┐   ┌───────┐   ┌───────┐
    │OpenAI │   │ Groq  │   │Google │
    │GPT-4o │   │ Qwen  │   │Gemini │
    └───────┘   └───────┘   └───────┘

Updated for LangChain 1.0+ (2025-2026)
"""

import os
from dotenv import load_dotenv

load_dotenv()

# Set up multiple provider API keys
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

print("✅ API keys configured for multiple providers")

In [ ]:
# =============================================================================
# Method 1: init_chat_model() - Universal Model Initialization
# =============================================================================
"""
init_chat_model() automatically detects the provider from the model name
and initializes the correct chat model class.

Supported model name patterns:
- OpenAI: "gpt-4o", "gpt-4o-mini", "gpt-4.1", "gpt-4.1-mini"
- Anthropic: "claude-3-opus", "claude-3-sonnet", "claude-3-haiku"
- Google: "gemini-pro", "gemini-1.5-pro"
- Groq: Requires explicit ChatGroq class

Benefits:
- No need to import provider-specific classes
- Consistent API across all providers
- Easy model switching
"""

from langchain.chat_models import init_chat_model
from langchain_groq import ChatGroq

# =============================================================================
# OpenAI Models via init_chat_model()
# =============================================================================
print("=" * 60)
print("Example 1: OpenAI with init_chat_model()")
print("=" * 60)

# Simple initialization - auto-detects OpenAI
openai_model = init_chat_model("gpt-4o-mini")

# Invoke with a simple question
response = openai_model.invoke("What is the capital of France?")
print(f"\n✅ OpenAI Response: {response.content}")
print(f"📊 Model Used: {response.response_metadata.get('model_name', 'unknown')}")
print(f"📈 Tokens: {response.response_metadata.get('token_usage', {})}")

# =============================================================================
# Groq Models (Ultra-Fast Inference)
# =============================================================================
print("\n" + "=" * 60)
print("Example 2: Groq (Fast Inference with Reasoning)")
print("=" * 60)

# Groq supports reasoning models like Qwen
groq_model = ChatGroq(
    model="qwen/qwen3-32b",      # Qwen 3 32B with reasoning
    temperature=0,               # Deterministic output
    max_tokens=None,             # No limit
    reasoning_format="parsed",   # Parse reasoning separately
    timeout=None,
    max_retries=2,
)

print("\n🚀 Groq is known for extremely fast inference speeds!")
print("   Supports: Llama, Mixtral, Qwen, and more")

In [ ]:
# =============================================================================
# Streaming Responses
# =============================================================================
"""
Streaming provides real-time output as the model generates text.
Useful for:
- Better user experience (see output immediately)
- Long responses
- Chatbots and interactive applications

Methods:
- .stream() - Yields chunks as they arrive
- .astream() - Async streaming
"""

print("=" * 60)
print("Example 3: Streaming with Groq")
print("=" * 60)
print("\n📝 Streaming response (each | is a new chunk):\n")

# Stream the response
for chunk in groq_model.stream("What is the capital of France? Tell me a fun fact about it."):
    # chunk.text contains the content (may be empty for reasoning chunks)
    if hasattr(chunk, 'text') and chunk.text:
        print(chunk.text, end="|", flush=True)
    elif hasattr(chunk, 'content') and chunk.content:
        print(chunk.content, end="|", flush=True)

print("\n\n✅ Streaming complete!")

# =============================================================================
# Comparing Streaming Output Formats
# =============================================================================
print("\n" + "=" * 60)
print("Example 4: OpenAI Streaming")
print("=" * 60)
print("\n📝 OpenAI streaming:\n")

for chunk in openai_model.stream("Name 3 programming languages"):
    if chunk.content:
        print(chunk.content, end="", flush=True)

print("\n")

In [ ]:
# =============================================================================
# Summary: Model Integration Key Points
# =============================================================================
"""
Model Integration - Quick Reference:
------------------------------------

✅ UNIVERSAL INITIALIZATION:
   from langchain.chat_models import init_chat_model
   model = init_chat_model("gpt-4o-mini")  # Auto-detects provider

✅ PROVIDER-SPECIFIC CLASSES:
   from langchain_openai import ChatOpenAI
   from langchain_groq import ChatGroq
   from langchain_google_genai import ChatGoogleGenerativeAI
   from langchain_anthropic import ChatAnthropic

✅ COMMON PARAMETERS:
   - model: Model name/ID
   - temperature: 0-2 (0 = deterministic, higher = creative)
   - max_tokens: Maximum output length
   - timeout: Request timeout
   - max_retries: Retry attempts

✅ INVOCATION METHODS:
   model.invoke("question")          # Single call
   model.stream("question")          # Streaming
   model.batch(["q1", "q2"])         # Multiple inputs
   await model.ainvoke("question")   # Async

✅ RESPONSE STRUCTURE:
   response.content              # The text response
   response.response_metadata    # Model info, tokens, etc.
   response.usage_metadata       # Token counts

Popular Models (2025-2026):
---------------------------
- OpenAI: gpt-4o, gpt-4o-mini, gpt-4.1, gpt-4.1-mini
- Anthropic: claude-3-opus, claude-3-sonnet, claude-3-haiku
- Google: gemini-1.5-pro, gemini-1.5-flash
- Groq: llama-3-70b, mixtral-8x7b, qwen/qwen3-32b

Pro Tips:
---------
- Use init_chat_model() for quick prototyping
- Use provider-specific classes for advanced features
- Set temperature=0 for deterministic outputs
- Use streaming for better UX in chat applications
"""

print("=" * 60)
print("📚 Summary: Model Integration")
print("=" * 60)
print("""
Key Methods:
- init_chat_model("model-name")  → Universal initialization
- model.invoke("prompt")         → Single call
- model.stream("prompt")         → Streaming output
- model.batch([...])             → Multiple inputs
""")